In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\Arpit Kadam\\Desktop\\DataScienceProject\\data-science-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH, 
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Create root directory
        create_directories([self.config["artifacts"]["artifacts_root"]])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config= self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )

        return model_trainer_config

In [20]:
import pandas as pd
import os
from src.datascienceproject import logger
from sklearn.linear_model import ElasticNet
import joblib


In [23]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        logger.info("Training the model and saving it")
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        x_train = train_data.drop(columns=[self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        x_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(x_train, y_train)
        logger.info("Model trained successfully")
        
        logger.info("Saving the model")
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

        logger.info(f"Model saved at {self.config.root_dir}/{self.config.model_name}")

        logger.info("Model training completed")

In [24]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(f"Training failed: {str(e)}")
    raise e 

[2024-12-26 18:00:55,183: INFO: common: Yaml file: config\config.yaml read successfully]
[2024-12-26 18:00:55,186: INFO: common: Yaml file: params.yaml read successfully]
[2024-12-26 18:00:55,191: INFO: common: Yaml file: schema.yaml read successfully]
[2024-12-26 18:00:55,194: INFO: common: Created directory at: ./artifacts]
[2024-12-26 18:00:55,195: INFO: common: Created directory at: ./artifacts/model_trainer]
[2024-12-26 18:00:55,197: INFO: 1409113851: Training the model and saving it]
[2024-12-26 18:00:55,212: INFO: 1409113851: Model trained successfully]
[2024-12-26 18:00:55,213: INFO: 1409113851: Saving the model]
[2024-12-26 18:00:55,224: INFO: 1409113851: Model saved at ./artifacts/model_trainer/model.joblib]
[2024-12-26 18:00:55,227: INFO: 1409113851: Model training completed]
